# Airbnb Pricing Analysis

# SUMMARY

## Project Goal

The goal of this project is to build a model to predict the rental prices of AirBNB listings based on various factors such as location, size, etc 

## Data Wrangling

1. Transform Date columns (host_since, first_review, last_review) to date Format

2. Transform dollar amount columns (price, weekly_price, monthly_price, security_deposit, cleaning_fee):
    a) removing $ and "," signs
    b) filling the empty dollar values with 0

3. Transform some columns with discrete values to type "category". These are property_type, room_type, bed_type, cancellation_policy

4. Run drop_duplicates() on dataframe to remove duplicates.

### Initial Review
1. Total Record Count: 7785<BR>
2. NULL Columns (ie with missing values):<BR>
|**Columns|Values Count**|<BR>
|bathrooms|7778|<BR>
|bedrooms|7784|<BR>
|square_feet|217|<BR>
|weekly_price|1302|<BR>
|monthly_price|1012|<BR>
|security_deposit|5563|<BR>
|cleaning_fee|6782|<BR>
|first_review|6724|<BR>
|last_review|6731|<BR>

## Exploratory Data Analysis (EDA)

**The analysis will mainly use these information:**
1. The rental configurations (property type, room type, bathrooms, bedrooms, square_feet, etc)
2. pricing (price, weekly_price, monthly_price, security_deposit, cleaning_fee) -> Discount (derived)
3. host's policy (guests_included, extra_people, minimum_nights, maximum_nights, cancellation_policy)
4. host (host_is_superhost)
5. Airbnb reviews (number_of_reviews,first_review, last_review)

# Actual Notebook

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#r_listings = pd.read_csv('sea_first100.csv', index_col='id');
r_listings_raw = pd.read_csv('sea_full.csv', index_col='id');

In [10]:
print(r_listings_raw.shape)
r_listings_raw.head(2)

(7785, 95)


,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,...,requires_license,license,jurisdiction_names,instant_bookable,is_business_travel_ready,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
id,,,,,,,,,,,,,,,,,,,,,
2318,https://www.airbnb.com/rooms/2318,20180517031042,2018-05-17,"Casa Madrona - Urban Oasis, 1 block from the P...","Gorgeous, architect remodeled, 1917 Dutch Colo...","Casa Madrona is a gorgeous, architect remodele...","Gorgeous, architect remodeled, 1917 Dutch Colo...",none,NaN,"We adhere to a 10pm -9am quiet hour schedule, ...",...,f,NaN,{WASHINGTON},t,f,strict_14_with_grace_period,f,f,2,0.14
4291,https://www.airbnb.com/rooms/4291,20180517031042,2018-05-17,Sunrise in Seattle Master Suite,This property is located in the lovely Maple L...,"Located in Seattle, this is a spacious, clean ...",This property is located in the lovely Maple L...,none,NaN,NaN,...,f,NaN,{WASHINGTON},f,f,moderate,f,f,5,0.91


In [11]:
# TODO: General (Ref Unit 7.1.2 video)
# 1) What's the story?
# 2) What's memorable example of this? (ref: Linda the Bank Teller) (6/30)
# 3) What're more questions this will raise?
# 4a) Who is your audience? (but should be accessible to broadest audiences possible)
# 4b) What questions are you answering?
# 4c) Why should the audience care?
# 4d) What are your major insights and surprises? What change to you want to affect?

# 1) Story: Being an Airbnb host is not easy. Besides dealing with changing local 
#            regulations on the rental and uncertainty of renting out your house, 
#            the host also needs to ensure there is profit after incurring all the costs. 
# 4a) Audience: Airbnb hosts
# 4b) Questions Answering:
#     1) What drives price? Loc (eg Zips) or Configs (bed_type, room_type, property_type)    
# 4c) Why they care: Help them maximize profit by optimize the rental price, weekly and monthly prices


## Data Cleaning

### Initial Review: List columns with Null values

In [12]:
# Initial Review
# 1) List columns with Null values in file (use info() to identify)

required_data = ['property_type','room_type','accommodates',
'bathrooms','bedrooms','beds',
'bed_type','square_feet','price',
'weekly_price','monthly_price','security_deposit',
'cleaning_fee','guests_included','extra_people',
'minimum_nights','maximum_nights','number_of_reviews', 'host_since',
'first_review','last_review','cancellation_policy']

r_listings = r_listings_raw[:][required_data]
print(r_listings.shape)
r_listings.info()

(7785, 22)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 7785 entries, 2318 to 25133573
Data columns (total 22 columns):
property_type          7785 non-null object
room_type              7785 non-null object
accommodates           7785 non-null int64
bathrooms              7778 non-null float64
bedrooms               7784 non-null float64
beds                   7785 non-null int64
bed_type               7785 non-null object
square_feet            217 non-null float64
price                  7785 non-null object
weekly_price           1302 non-null object
monthly_price          1012 non-null object
security_deposit       5563 non-null object
cleaning_fee           6782 non-null object
guests_included        7785 non-null int64
extra_people           7785 non-null object
minimum_nights         7785 non-null int64
maximum_nights         7785 non-null int64
number_of_reviews      7785 non-null int64
host_since             7779 non-null object
first_review           6724 non-null object

### Initial Review: Visual review of Categorical Data (eg property/room/bed types) for bad values

In [13]:
# Review: Categories data
print("property_type")
print(r_listings['property_type'].value_counts(dropna=False))
print("\nroom_type")
print(r_listings['room_type'].value_counts(dropna=False))
print("\nbed_type")
print(r_listings['bed_type'].value_counts(dropna=False))

property_type
House                 2879
Apartment             2806
Townhouse              611
Guest suite            475
Condominium            340
Guesthouse             174
Serviced apartment      98
Loft                    94
Bungalow                85
Other                   43
Bed and breakfast       41
Cottage                 25
Boat                    21
Cabin                   19
Camper/RV               15
Houseboat               12
Tiny house              10
Hostel                   9
Tent                     7
Castle                   3
Resort                   3
Villa                    3
Boutique hotel           2
In-law                   2
Earth house              1
Farm stay                1
Hut                      1
Chalet                   1
Treehouse                1
Yurt                     1
Nature lodge             1
Lighthouse               1
Name: property_type, dtype: int64

room_type
Entire home/apt    5663
Private room       1967
Shared room         155
Name:

### Data Cleaning of Numeric Data

In [14]:
# Numeric Columns
# 1) Remove $ sign for columns 
# 2) Fill Null values with 0
# Ref: Cleaning Data (UNIT 5.1.4_ Using regular expressions to clean string)

def remove_sign(amt):
   amt = amt.replace('$', '').replace(',','')
   return(float(amt))

# TODO: 7/3/18:
# 1) just use groupby by zipcode to use mean(), etc. (Make sure blank is NaN and no String values)

# Fill null values with 0
r_listings['price'] = r_listings['price'].fillna('0') 
r_listings['weekly_price'] = r_listings['weekly_price'].fillna('0') 
r_listings['monthly_price'] = r_listings['monthly_price'].fillna('0') 
r_listings['security_deposit'] = r_listings['security_deposit'].fillna('0') 
r_listings['cleaning_fee'] = r_listings['cleaning_fee'].fillna('0') 

# Strip $ or "," signs from amt values
r_listings['price']  = r_listings['price'].apply(remove_sign)
r_listings['weekly_price']  = r_listings['weekly_price'].apply(remove_sign)
r_listings['monthly_price']  = r_listings['monthly_price'].apply(remove_sign)
r_listings['security_deposit']  = r_listings['security_deposit'].apply(remove_sign)
r_listings['cleaning_fee']  = r_listings['cleaning_fee'].apply(remove_sign)

### Visual review of Numeric Data for Outliers

In [15]:
# Review: Numeric data
print('Summary of price')
print(r_listings['price'].describe())

# print("\nMore Bathrooms than bedrooms")

Summary of price
count     7785.000000
mean       150.853821
std        255.345205
min          0.000000
25%         77.000000
50%        110.000000
75%        175.000000
max      10000.000000
Name: price, dtype: float64


### Cleaning and tidying datetime data

In [16]:
# Ref 5.1.1
# 6/26/18: Parse Date columns
r_listings['host_since'] = pd.to_datetime(r_listings['host_since'])
r_listings['first_review'] = pd.to_datetime(r_listings['first_review'])
r_listings['last_review'] = pd.to_datetime(r_listings['last_review'])

In [17]:
# TODO: Cleaning numeric data (Ref: 5.1.1)
# 1) Identify columns should be numeric, use dtypes() after load; if type is "object" -> non-numeric values in data
# 2) Use pd.to_numeric(), ref: "Cleaining Numberic column" exercise
#    a) Also Unit 5.1.4 start, use para errors='coerce' (turn bad values to "NaN")


### Transform Categorical Data

In [18]:
# TODO: Misc Cleaning (6/29/18)
# 0) Use count() to review columns that have NULL data
# 1) Identify columns that should be Categorical, then change the dtype
#    eg tips.smoker = tips.smoker.astype('category')
# 2) Identify columns with Missing Data
# 3) Run df.drop_duplicates() to ensure no dups
# 4) MIssing Data: If needed, (ref: Unit 5.1.4 chap4)
#    a) use tips_nan.dropna() to drop them or 
#    b) fillna() to fill them

# TODO (7/3/18): 
#    1) convert the type values to numeric values so modeling like random forest; etc
#    2) Linear Regression: Can't pass number; one hot encoding / dummy variables.

# 1) Identify columns that should be Categorical, then change the dtype
# columns: property_type, room_type, bed_type, cancellation_policy
r_listings['property_type'] = r_listings['property_type'].astype('category')
r_listings['room_type'] = r_listings['room_type'].astype('category')
r_listings['bed_type'] = r_listings['bed_type'].astype('category')
r_listings['cancellation_policy'] = r_listings['cancellation_policy'].astype('category')

r_listings.drop_duplicates() 
print("Done with cleaning")  # 7/24/18: drop_duplicates() somehow lists out whole dataframe

Done with cleaning


##   Exploratory Data Analysis (EDA) 

## Questions I Ask
### Num of Bedrooms vs price
ScatterPlots

### Number of listings vs price
Histogram
a) Y-axis: Number of listing;  
b) X-axis: Break the price ranges to 10 buckets  (eg $0 - $100, $100-$200, $200-$300) and get Average price
c) X-axis: 2 levels: Outer layer - Num of bedrooms; Inner Layer - Average Price in each bucket

### ZIPs vs Avg Price by ZIP
Bar Chart

## Guideline
Steps:
1. Ask the following questions and look for the answers using code and plots:<BR>
Can you count something interesting?<BR>
Can you find trends (e.g. high, low, increasing, decreasing, anomalies)?<BR>
Can you make a bar plot or a histogram?<BR>
Can you compare two related quantities?<BR>
Can you make a scatterplot?<BR>
Can you make a time-series plot?<BR>
    
2. Looking at the plots, what are some insights you can make? Do you see any correlations? Is there a hypothesis you’d like to investigate further? What other questions do the insights lead you to ask?

3. Now that you’ve asked questions, hopefully you’ve found some interesting insights. Is there a narrative or a way of presenting the insights using text and plots that tells a compelling story? What are some other trends/relationships you think will make the story more complete?

## The Notebook should contain:
1. The questions you asked
2. The trends you investigated
3. The resulting visualizations and conclusions

In [19]:
# TODO: Ensure Effective Visualizations (6/29/18)
# 1) Ensure not too crowded, clustered. (ref: 03-EDA.pdf)
# 2) Bad ones: spend time to read legions, think hard...
# 3) Good ones:
#   a) Have graphical integrity 2. Keep it simple
#   b) Use the right display
#   c) Use color strategically 5. Tell a story with data
# 4) Guidelines:
#   a) Bar chart: should start from 0; if smaller changes -> use other plots (dot plots)


In [20]:
# UNIT 8.5.1: Project: Capstone Project 1: Exploratory Data Analysis
# REQUIREMENTS:
# 1) Are there variables that are particularly significant in terms of explaining the answer to your project question?
# 2) Are there strong correlations between pairs of independent variables or between an independent and a dependent variable?
# 3) What are the most appropriate tests to use to analyse these relationships?

## ---------------
# 1) Variables: 
#    a) zipcode; bathroom, bedroom, square feet; types (property_type, room_type, bed_type) 
#    b) Prices (price, weekly, monthly); 
#    c) cancellation_policy, security_deposit, cleaning_fee


In [21]:
# TODOs: 7/1/18:
# 1) Hist: Y:Zip, X:Avg Price by ZiP
# 1b) Hist: above but break down by bed_type, room_type
# 2) Plotter: similar set 
# TBD: Watch out for outliers

# References:
# 1) Art of Effective Visualization of Multi-dimensional Data
#   URL: https://towardsdatascience.com/the-art-of-effective-visualization-of-multi-dimensional-data-6c7202990c57

In [22]:
# CHART: ScatterPlots: x: num of bedrooms, y: price
# Conclusion: 1) NO correlation between Number of Bedrooms vs Prices
# 7/7/18: Mentor suggests to use Histogram instead.

# TODOs/QUESTIONS:
# 1) How to plot 2 groups on 2 room types (Entire home/apt vs Private room)
# 2) What to do with few outliners that expand prices scale from 2000 or more?

# Misc Notes:
# a) r_listings['bedrooms']: type is Pandas Series
# b) Needed to ensure prices is numeric so will be shown in order

# 7/2/18: Control size of the in-line charts
# plt.rcParams['figure.figsize'] = [15, 15]

# TODOs: 7/3/18 (after mentor call)
# 1) Use sns.FacetGrid, put 2 diff colors (Use DJ's article on Visualization)
#   a) search for "fig.suptitle('Wine Type - Alcohol - Quality - Acidity', fontsize=14)"
#   b) "jp = sns.pairplot(wines, x_vars=["sulphates"], y_vars=["alcohol"], size=4.5,"

# plt.scatter(r_listings['bedrooms'], r_listings['price']) # scatter
# plt.title('Number of Bedrooms vs Prices')
# plt.xlabel('Num of Bedrooms')
# plt.ylabel('Prices')
# plt.show()

In [23]:
# CHART: Histogram: (7/7/18)
# a) Y-axis: Number of listing; 
# b) X-axis: Break the price ranges to 10 buckets  (eg $0 - $100, $100-$200, $200-$300) and get Average price
# c) X-axis: 2 levels: Outer layer - Num of bedrooms; Inner Layer - Average Price in each bucket

# Ref: Unit 5.1.2: Hierarchical.. Index (for 2-level X-axis)
# eg. stocks = stocks.set_index(['Symbol', 'Date'])

In [24]:
# CHART: Bar Chart1 
# C1: Hist: Y:Zip, X:Avg Price by ZiP
# C2: Hist: Y:Zip, X:Avg Price by ZiP but break down by bed_type, room_type, property_type

# Conclusion: TBD

# TODOs/QUESTIONS:
# 1) 7/3/18: too hard to see all zip codes. Any suggestions?
# 2) 7/5/18: How do I sort the zipcodes?

# NOTES: 7/7/18
# 1) Set zipcodes as Index and use sort_index() so appear sorted (Unit 5.1.2)

# NOTES: 7/3/18 (after mentor call)
# 2) (Test) use func (https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html)
# 3) Use Seaborn (https://seaborn.pydata.org/generated/seaborn.barplot.html)
# 4) When I have zipcode on Y-axis, it's harder to see whole chart but easier to see the zipcode
sns.set_style("whitegrid")
ax = sns.barplot(y="zipcode", x="price", hue="room_type", data=r_listings) # break down by room_types
#ax = sns.barplot(y="price", x="zipcode", data=r_listings) # work1
plt.title('Prices per Zipcodes by Room Types')
plt.xlabel('Zipcodes')
plt.ylabel('Prices')
plt.show()

# Ver1: Work1
# =========
# price_by_zips = r_listings.groupby('zipcode').price.mean()
# deposit_by_zips = r_listings.groupby('security_deposit').price.mean()
# print(price_by_zips.head(8))
# price_by_zips.plot(x='zipcode', kind='bar') 
# price_by_zips.plot(x='security_deposit', kind='bar') 
# plt.show()

# NOTES: 7/3/18: Histogram is for continus values but by zipcodes in here is discrete so WRONG chart type to use. 

ValueError: Could not interpret input 'zipcode'

In [ ]:
# CHART: Bar Chart 2

# TODO/Questions: 7/5/18
# 1) Suggestions: Many property types make it hard to read and crowded, other ways?
# 2) Outliers: Castle, how to address?

plt.subplot(2,1,1) # Grid row, Grid, column, current subplot 
sns.set_style("whitegrid")
price_by_type = sns.barplot(y="property_type", x="price", data=r_listings) # break down by room_types

plt.subplot(2,1,2)
sns.set_style("whitegrid")
cnt_by_type = sns.countplot(x="property_type", data=r_listings) 
plt.show()

# TODOs: 7/3/18 (after mentor call)
# 1) Try to use Bar plot instead, only use Pie chart for few categories

In [ ]:
# CHART: Histogram 1 
# 1) X - price per square foot, Y - count (ref: UNIT 10.1.2 NY Housing) (7/5/18)
# 2) Take a log() to see difference

In [ ]:
# CHART: Pearson Correlation
# TODOs: 7/3/18 (after mentor call)
#  1) https://seaborn.pydata.org/generated/seaborn.heatmap.html (Example at end of page)
#  2) Check https://seaborn.pydata.org/examples/many_pairwise_correlations.html

# SAMPLE: Pearson Correlation
# corr = np.corrcoef(np.random.randn(10, 200))
# mask = np.zeros_like(corr)
# mask[np.triu_indices_from(mask)] = True
# with sns.axes_style("white"):
#      ax = sns.heatmap(corr, mask=mask, vmax=.3, square=True)

In [ ]:
##  =================== Features Engineering ===================

In [ ]:
# TODOs: (as of 7/5/18)
# 1) Assign ranking based on bed_type, room_type and property_type? (7/1/18)
# 2) Price/Sq foot, only consider price is meaningless (7/5/18)

In [ ]:
# TODO: Machine Learning (6/30/18) (Unit 10.1)
# 1) Any bias in data? Eg from un-collected data

In [ ]:
##  =================== Modeling ===================

In [ ]:
## References:
# 1) UNIT 10.1.2: NY Housing Example 
#    a) Use value/sq foot

In [ ]:
## Model1: Linear Regressions

In [ ]:
## Model2: K-nearest neighbors

In [ ]:
# QUESTIONS (7/3/18):
# 3) K-nearest neighbors: 
#     a) Is using Num of bedrooms make most sense?
#     b) OR better to use ZIP, prop_type, etc? How to determine best attribute?